In [3]:
from langchain_community.document_loaders import PyPDFLoader

file_path = r"D:\GenAI\RAG\data\ffiec_itbooklet_developmentacquisitionmaintenance.pdf"
loader = PyPDFLoader(file_path)

docs = loader.load()

print(len(docs))

222


In [ ]:
print(f"{docs[140].page_content[:150]}\n")
print(docs[140].metadata)

In [4]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)

len(all_splits)

905

In [5]:
from openai import AzureOpenAI
from dotenv import load_dotenv
import os
import getpass

In [6]:
load_dotenv()
AZURE_OPENAI_API_KEY = os.getenv("OPEN_API_KEY")
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")

In [7]:
from langchain_openai import AzureOpenAIEmbeddings

embeddings = AzureOpenAIEmbeddings(
    model="text-embedding-3-small",
    # dimensions: Optional[int] = None, # Can specify dimensions with new text-embedding-3 models
    # azure_endpoint="https://<your-endpoint>.openai.azure.com/", If not provided, will read env variable AZURE_OPENAI_ENDPOINT
    # api_key=... # Can provide an API key directly. If missing read env variable AZURE_OPENAI_API_KEY
    # openai_api_version=..., # If not provided, will read env variable AZURE_OPENAI_API_VERSION
)

In [8]:
vector_1 = embeddings.embed_query(all_splits[0].page_content)
vector_2 = embeddings.embed_query(all_splits[1].page_content)

assert len(vector_1) == len(vector_2)
print(f"Generated vectors of length {len(vector_1)}\n")
print(vector_1[:10])

Generated vectors of length 1536

[-0.010430463589727879, -0.005864460486918688, 0.039866916835308075, -0.00943165086209774, 0.029621947556734085, -0.013712278567254543, 0.0031979859340935946, 0.009845444932579994, -0.0033281883224844933, 0.015923935920000076]


In [9]:
from langchain_community.vectorstores import Chroma

vector_store = Chroma(embedding_function=embeddings)

C:\Users\kusha\AppData\Local\Temp\ipykernel_20808\2269285310.py:3: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vector_store = Chroma(embedding_function=embeddings)


In [11]:
ids = vector_store.add_documents(documents=all_splits)

In [13]:
query = "What are the minimum security controls required for the development of IT systems in the document?"

In [17]:
results = vector_store.similarity_search_with_score(
    query,
    k=10
)



In [18]:
for doc, score in results:
    print(f"Content: {doc.page_content}")
    print(f"Metadata: {doc.metadata}")
    print(f"Similarity Score: {score}")  # The lower the score, the more similar
    print("---")

Content: FFIEC IT Examination Handbook  Development, Acquisition, and Maintenance 
August 2024 95 
efficiently. Finally, use of naming conventions often facilitates software code reuse and 
repurposing. 
 
To establish development standards and controls, prudent practices include thorough 
development testing and validation of systems and component-related code. Effective 
management keeps completed systems and component-related code that have passed security 
certification in program libraries as discussed in the “Additional Control Considerations in 
Change Management” section of this booklet. 
 
Due to supply chain risk in development, for both in-house and supply chain partners, it is 
important to consider using the following standards and controls when developing information 
systems and components: 
 
• Design security controls to be difficult to disable (e.g., tamper-proofing techniques), and, if 
they are disabled, trigger notification methods such as audit trails, tamper evid